# Prepare

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 4.4 MB/s eta 0:00:00


In [3]:
%cd '/content/drive/MyDrive/BDT SEM2/5005/'

/content/drive/MyDrive/BDT SEM2/5005


In [4]:
# os
import os
# storage
import pickle

# data processing
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np

# visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# utils
from random import sample
import datetime as dt
from collections import Counter

# tqdm
from tqdm import tqdm

# warnings
import warnings

warnings.filterwarnings("ignore")

# Convert every .pkl to .csv and remove unnecessary columns

In [ ]:
cols=[\
      'id',\
      'date',\
      'username',\
      'rawContent',\
      'retweetCount',\
      'likeCount'
      ]

def pkl_to_df(tweets_dict):
    data = []
    for tid, tweet in tweets_dict.items():
        twi = list()
        # tweet id
        twi.append(tweet.id)
        # date
        twi.append(tweet.date.strftime('%Y-%m-%d'))
        # usr name
        twi.append(tweet.user.username)
        # content
        twi.append(tweet.rawContent)
        # retweetCount
        twi.append(tweet.retweetCount)
        # likeCount
        twi.append(tweet.likeCount)
        
        data.append(twi)
    return pd.DataFrame(data, columns=cols)

In [ ]:
files = tqdm(os.listdir('./dataset/'))
target_dir = './dataset_rawcontent/'
for file in files:
    if file.endswith('.pkl'):
        with open('./dataset/'+file, 'rb') as f:
            tweets_dict = pickle.load(f)
            tweets_df = pkl_to_df(tweets_dict)
            if not os.path.exists(target_dir):
                os.mkdir(target_dir)
            tweets_df.to_csv(target_dir+file.strip('.pkl')+'.csv', index=False)
        

100%|██████████| 21/21 [06:38<00:00, 18.99s/it]


# Combine all csv and drop duplicate tweets

In [ ]:
%cd './dataset_rawcontent/'

/content/drive/MyDrive/BDT SEM2/5005/dataset_rawcontent


In [ ]:
csvfiles = tqdm(os.listdir())
target_dir = './dataset_rawcontent/'
df_l = []
for file in csvfiles:
    if file.endswith('.csv') and (not file.startswith('SUM')):
        df_l.append(pd.read_csv(file))
concat_df = pd.concat(df_l, ignore_index=True)

100%|██████████| 20/20 [00:18<00:00,  1.08it/s]


In [ ]:
concat_df = concat_df.drop_duplicates(subset=['id'])
concat_df = concat_df.drop_duplicates(subset=['rawContent'])
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71712 entries, 0 to 1202619
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            71711 non-null  float64
 1   date          71711 non-null  object 
 2   username      71711 non-null  object 
 3   rawContent    71711 non-null  object 
 4   retweetCount  71711 non-null  float64
 5   likeCount     71711 non-null  float64
 6   Unnamed: 0    1 non-null      float64
 7   word          1 non-null      object 
 8   tfidf         1 non-null      float64
dtypes: float64(5), object(4)
memory usage: 5.5+ MB


In [ ]:
concat_df.to_csv('SUM.csv', index=False)

# TF-IDF

In [ ]:
%cd './dataset_rawcontent/'

/content/drive/MyDrive/BDT SEM2/5005/dataset_rawcontent


In [ ]:
tif_df = pd.read_csv('sum_processed.csv')
tmp = tif_df.lemmaContent
p_corpus = tmp.dropna().tolist()
p_corpus

['look speak worker start ai model help job tell boss happy talk anonymously',
 'ask     damn self want read',
 'mute     kindness fuck ask',
 'zero ai     marketing thing datum     neural map     map parse     unreal speed intelligence',
 'fuck end mix detail news conference         sound like write story shit',
 'congrat find combination copilot      powerful hack prototype new company quickly touch lot technology know deeply excited experience come',
 'machine learn artificial intelligence manipulate datum datum magic',
 'opinion     tool weapon chit juan read',
 '      task administrative       legal profession automate      construction      maintenance uncertainty long term impact ai     ai tech job jobcut',
 'goldman sach report suggest ai replace       million job globally bring productivity boom new job ai ability replace quarter work task europe generative ai particularly see major advancement     tech',
 '    rule odd historian know include city hall record keeper cover chro

In [ ]:
for s in p_corpus:
    if 'datum' in p_corpus:
        print(s)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X=vectorizer.fit_transform(p_corpus)

In [ ]:
data = {'word':vectorizer.get_feature_names_out(), 'tfidf':X.toarray().sum(axis=0).tolist()}
X_df = pd.DataFrame(data)
X_df = X_df.sort_values(by='tfidf', ascending=False)
X_df

,word,tfidf
657,ai,2431.133786
33754,write,1377.711780
1940,ask,1084.910469
17524,like,1062.612066
12677,google,1032.717092
...,...,...
7325,daaa,0.093023
9188,dunnn,0.093023
28609,stargazing,0.084429
2623,barbecuing,0.084429


In [ ]:
X_df.to_csv('TFIDF_words_lemm.csv')

# Topic Modeling

In [5]:
!pip install contractions
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 10.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 48.7 MB/s eta 0:00:00


In [6]:
import pandas as pd
import unicodedata
import re
import contractions
import string
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#spacy
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
#vis
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Clean & Preprocessing & Save

### Functions:

**Lower-Case Conversion:**

convert our entire text data into lowercase.

In [ ]:
# Lower-Case Conversion
def to_lowercase(text):
  return text.lower()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Standardizing Accent Characters:**

Sometimes, people use accented characters like é, ö, etc. to signify emphasis on a particular letter during pronunciation. In some instances, accent marks also clarify the meanings of words, which might be different without accent marks. Though the use of accent marks is very limited, it’s a good practice to convert these characters into standard ASCII characters.

In [ ]:
def standardize_accented_chars(text):
  return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  
#testing the function on a single sample for explaination
print(standardize_accented_chars('Sómě words such as résumé, café, prótest, divorcé, coördinate, exposé, latté.'))

Some words such as resume, cafe, protest, divorce, coordinate, expose, latte.


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Removing URLs**

In [ ]:
def remove_url(text):
  return re.sub(r'https?:\S*', '', text)

#testing the function on a single sample for explaination
print(remove_url('using https://www.google.com/ as an example'))
print(remove_url('using http://www.google.com/ as an example'))

using  as an example
using  as an example


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Expanding Contractions**

Contractions are shortened versions of words or syllables. They are created by removing, one or more letters from words. More often than not, multiple words are combined to create a contraction. For example, I will is contracted into I’ll, do not into don’t. We wouldn’t want our model to consider I will and I’ll differently. Hence, we will convert each contraction into its expanded form using the below-mentioned code.

In [ ]:
def expand_contractions(text):
  expanded_words = [] 
  for word in text.split():
    expanded_words.append(contractions.fix(word)) 
  return ' '.join(expanded_words)

#testing the function on a single sample for explaination
print(expand_contractions("Don't is same as do not"))
print(expand_contractions("I am gonna go"))
print(expand_contractions("I'll be there within 5 min. Shouldn't you be there too?\
I'd love to see u there my dear. It's awesome to meet new friends.\
We've been waiting for this day for so long."))

Do not is same as do not
I am going to go
I will be there within 5 min. Should not you be there too?I would love to see you there my dear. It is awesome to meet new friends.We have been waiting for this day for so long.


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Removing Mentions and Hashtags**

After a quick analysis, I found that in this example, mentions and hashtags don’t contribute much to the information relevant for topic modeling as they don’t follow any pattern i.e. they are randomly used in tweets. Remove mentions and tags using the function mentioned below.

In [ ]:
def remove_mentions_and_tags(text, rm_mention=True, rm_tag=True):
  if rm_mention:
    text = re.sub(r'@\S*', '', text)
  if rm_tag:
    text = re.sub(r'#\S*', '', text)
  return text

#testing the function on a single sample for explaination
print(remove_mentions_and_tags('Some random @abc and #def'))

Some random  and 


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Keeping only Alphabet:** 

Punctuations, numbers, and special characters like ‘$, %, etc.’ don’t provide any information.

In [ ]:
def keep_only_alphabet(text):
  # if 'to_lowercase' already done:
  #text = re.sub(r'[^a-z]', ' ', text)
  # otherwise
  text = re.sub(r'[^A-z]', ' ', text)
  return text

#testing the function on a single sample for explaination
print(keep_only_alphabet('Just a bit more $$processing required.Just a #bit!!!'))

Just a bit more   processing required Just a  bit   


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Removing Stopwords(Default+Custom) and Removing Short Words:**

In [ ]:
#creating a spaCy object. 
nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])
my_stop_words = {'gpt','fuck','chatgpt', 'chatgpts', 'hey', 'hi', 'hmm', 'yo',\
'really','sometimes', 'go', 'since', 'whither', 'they', 'its', 'them',\
'well', 'meanwhile', 'seems', 'and', 'latterly', 'regarding', 'somehow',\
'sixty', 'whole', 'anyway', 'else', 'few', "'m", 'beside', 'to', 'namely',\
'someone', 'see', 'moreover', 'wherein', 'for', 'former', 'bottom', 'it',\
'next', 'six', 'along', 'once', 'might', 'whenever', 'below', 'another',\
'yourself', 'each', 'just', 'ourselves', 'everyone', 'any', 'across', \
'get', 'that', 'eight', 'we', 'which', 'therefore', 'may', "'s", 'keep',\
'among', 'give', 'such', 'are', 'indeed', 'everywhere', 'same', 'herself',\
'yourselves', 'alone', 'were', 'was', 'take', 'seem', 'say', 'why', 'show',\
'between', 'during', 'elsewhere', 'or', 'though', 'forty', 'made', 'used',\
'others', 'whereafter', 'formerly', 'several', 'via', 'does', 'please', 'three',\
'also', 'fifty', 'afterwards', "'s", 'noone', 'do', 'perhaps', 'further',\
'i', 'beforehand', 'myself', 'empty', "'ll", 'yet', 'thereby', 'been', 'both',\
'never', 'put', 'without', 'him', 'a', 'nothing', 'thereafter', 'make', 'then',\
'whom', 'must', 'sometime', 'against', 'through', 'being', 'four', 'back', 'become',\
'our', 'himself', 'because', 'anything', "'re", 'nor', 'therein', 'due', 'until',\
'own', 'ca', 'most', 'now', 'while', 'of', 'only', 'am', 'itself', 'too', "'m",\
'nobody', 'if', 'one', 'whereas', 'twelve', 'together', 'can', 'who', 'even',\
'be', 'she', 'besides', 'herein', 'off', "'d", 'last', 'no', 'whereupon', 'the', "'m",\
'thru', 'out', 'hereupon', 'by', 'us', 'already', 'became', 'here', 'hers', 'onto',\
'beyond', 'down', 'enough', 'did', 'some', 'over', 'serious', 'quite', 'move', 'around',\
'nowhere', 'amongst', 'but', 'so', 'wherever', 'twenty', 'often', 'part',\
'again', 'where', 're', 'within', 'at', "n't", 'yours', 'front', 'unless',\
'could', 'anyone', 'third', 'whatever', 'doing', "'d", 'nevertheless',\
'before', 'rather', 'fifteen', 'her', 'me', 'thereupon', 'mostly',\
'throughout', 'hence', "'re", 'mine', 'ten', 'hundred', 'nine', 'call',\
'when', 'about', 'will', 'whereby', 'this', 'upon', 'you', 'should',\
'always', 'themselves', 'not', 'has', 'behind', 'on', 'anywhere', 'side',\
'their', 'hereby', 'latter', 'after', "'ve", 'none', 'these', 'name',\
"n't", 'every', 'although', "'s", 'however', 'he', 'becoming', 'how', \
'whose', 'still', 'hereafter', 'whether', 'towards', 'more', 'everything',\
'whoever', 'seemed', 'cannot', 'up', 'otherwise', 'in', 'would', 'under',\
'done', 'thence', 'whence', 'seeming', 'either', 'other', 'with', 'into',\
'amount', 'five', 'much', "'re", 'except', 'his', 'thus', "'ll", 'what',\
'almost', 'becomes', 'least', 'ever', 'above', 'is', 'first', 'there',\
'somewhere', 'top', "'ve", 'than', "n't", 'have', 'toward', 'per', 'all',\
'ours', 'full', "'d", 'anyhow', 'as', "'ll", 'many', 'various', 'your',\
'had', 'eleven', 'from', 'something', 'less', 'those', 'using', 'an',\
'two', 'my', 'very', 'neither'}

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def remove_stopwords(text,nlp,custom_stop_words=None,remove_small_tokens=True,min_len=1):
  # if custom stop words are provided, then add them to default stop words list
  if custom_stop_words:
    nlp.Defaults.stop_words = custom_stop_words
  
  filtered_sentence = [] 
  doc = nlp(text)
  for token in doc:
    if token.is_stop == False: 
      # if small tokens have to be removed, then select only those which are longer than the min_len 
      if remove_small_tokens:
        if len(token.text) > min_len:
          filtered_sentence.append(token.text)
      else:
        filtered_sentence.append(token.text)

    # if after the stop word removal, words are still left in the sentence, then return the sentence as a string else return null 
  return ' '.join(filtered_sentence) if len(filtered_sentence) > 0 else np.nan

#removing stop-words and short words from every row
# tweets_df.Tweets=tweets_df.Tweets.apply(lambda x:remove_stopwords(x,nlp,{"elon","musk",}))
remove_stopwords(" chatgpt really say   socially liberal   fiscally conservative",nlp, my_stop_words)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'chatgpt    socially liberal    fiscally conservative'

**Lemmatization:**
Lemmatization generates the root of the word. It makes use of vocabulary and morphological analysis of words, to generate the root form of a word. We will use the spaCy library for performing lemmatization.

In [ ]:
def lemmatize(text, nlp):
  doc = nlp(text)
  lemmatized_text = []
  for token in doc:
    lemmatized_text.append(token.lemma_)
  return ' '.join(lemmatized_text)

#testing the function on a single sample for explaination
print(lemmatize('Reading NLP blog is fun.' ,nlp ))

#Performing lemmatization on every row
#tweets_df.Tweets=tweets_df.Tweets.apply(lambda x:lemmatize(x,nlp)

read NLP blog be fun .


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### preprocessing:

In [ ]:
%pwd

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'/content/drive/MyDrive/BDT SEM2/5005'

In [ ]:
%cd '/content/drive/MyDrive/BDT SEM2/5005/'

/content/drive/MyDrive/BDT SEM2/5005


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
t_df = pd.read_csv('/content/drive/MyDrive/BDT SEM2/5005/dataset_rawcontent/tweet_230201-230228.csv')
t_df.info(10)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225140 entries, 0 to 1225139
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            831924 non-null  object 
 1   date          831921 non-null  object 
 2   username      831921 non-null  object 
 3   rawContent    831897 non-null  object 
 4   retweetCount  515724 non-null  float64
 5   likeCount     515724 non-null  float64
dtypes: float64(2), object(4)
memory usage: 56.1+ MB


In [ ]:
#files = tqdm(os.listdir('./dataset_rawcontent/'))
files = tqdm(['tweet_230101-230130_az.csv'])
target_dir = './dataset_processed/'
for file in files:
    if file.endswith('.csv') and ('tweet' in file):
        # read
        df = pd.read_csv('./dataset_rawcontent/'+file)

        df['rawContent'] = df['rawContent'].astype('str') 

        tweets_df = df.rawContent.copy()

        # apply all the funcs
        tweets_df = tweets_df\
        .apply(to_lowercase)\
        .apply(standardize_accented_chars)\
        .apply(remove_url)\
        .apply(expand_contractions)\
        .apply(remove_mentions_and_tags)\
        .apply(keep_only_alphabet)

        tweets_df_rms = tweets_df.apply(lambda x:remove_stopwords(x,nlp,my_stop_words))

        tweets_df_lemma = tweets_df_rms.apply(lambda x:lemmatize(x,nlp) if x is not np.nan else np.nan)

        df['lemmaContent'] = tweets_df_lemma
        df['processedContent'] = tweets_df

        # save
        if not os.path.exists(target_dir):
            os.mkdir(target_dir)
        df.to_csv(target_dir+file, index=False)

## Pipeline
**start to process our data:**

first, define the text you want to perform topic modeling:

In [ ]:
%pwd

'/content/drive/MyDrive/BDT SEM2/5005'

In [ ]:
root = './dataset_processed/'
csvfiles = tqdm(os.listdir(root))
df_l = []
for file in csvfiles:
    if file.endswith('.csv') and file.startswith('tweet_verified'):
        df_l.append(pd.read_csv(root+file))
tweets_df = pd.concat(df_l, ignore_index=True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 18/18 [00:14<00:00,  1.20it/s]


filter

In [ ]:
#tweets_df = tweets_df.loc[df['likeCount'] > 2]
#filtered_df = tweets_df.loc[tweets_df['likeCount'] > 10]
filtered_df = tweets_df.copy()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
filtered_df.info()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202619 entries, 0 to 1202618
Data columns (total 8 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   id                1202619 non-null  int64 
 1   date              1202619 non-null  object
 2   username          1202619 non-null  object
 3   rawContent        1202619 non-null  object
 4   retweetCount      1202619 non-null  int64 
 5   likeCount         1202619 non-null  int64 
 6   lemmaContent      1202272 non-null  object
 7   processedContent  1202619 non-null  object
dtypes: int64(3), object(5)
memory usage: 73.4+ MB


In [ ]:
tweets_df_rms = filtered_df.lemmaContent.astype(str).apply(lambda x:remove_stopwords(x,nlp,my_stop_words))

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
filtered_df['rmsContent'] = tweets_df_rms
filtered_df

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,date,username,rawContent,retweetCount,likeCount,lemmaContent,processedContent,rmsContent
0,1640881210586193921,2023-03-29,cameronwilson,I'm looking to speak to workers who've started...,1,1,look speak worker start chatgpt ai model help ...,i am looking to speak to workers who have star...,look speak worker start ai model help job tell...
1,1640880822990557184,2023-03-29,katiewalshstx,"""I asked ChatGPT to-"" keep to your damn self, ...",0,2,ask chatgpt damn self want read,i asked chatgpt to keep to your damn self ...,ask damn self want read
2,1640880099749937152,2023-03-29,katiewalshstx,"I have to mute ""ChatGPT,"" with kindness, I cou...",3,18,mute chatgpt kindness fuck ask,i have to mute chatgpt with kindness i cou...,mute kindness ask
3,1640879874473689089,2023-03-29,martypartymusic,@Blacklist_Ray No there is zero AI in chatGPT ...,0,0,zero ai chatgpt marketing thing datum ...,no there is zero ai in chatgpt its a market...,zero ai marketing thing datum neural...
4,1640879019020230658,2023-03-29,WRBolen,How the fuck do you end up mixing in details f...,0,2,fuck end mix detail news conference so...,how the fuck do you end up mixing in details f...,end mix detail news conference sound ...
...,...,...,...,...,...,...,...,...,...
1202614,1598036175285276672,2022-11-30,sethbannon,Trying out ChatGPT and wow https://t.co/TQ0t7B...,15,163,try chatgpt wow,trying out chatgpt and wow,try wow
1202615,1598027429909778432,2022-11-30,minimaxir,ChatGPT thwarts my many attempts at malicious ...,22,186,chatgpt thwart attempt malicious javascript in...,chatgpt thwarts my many attempts at malicious ...,thwart attempt malicious javascript injection
1202616,1598024131978547200,2022-11-30,minimaxir,ChatGPT vs. riddles https://t.co/YuPYOD0U7o,2,18,chatgpt vs riddle,chatgpt vs riddles,vs riddle
1202617,1598015627540635648,2022-11-30,gdb,"Just launched ChatGPT, our new AI system which...",367,2361,launch chatgpt new ai system optimize dialogue...,just launched chatgpt our new ai system which...,launch new ai system optimize dialogue try


In [ ]:
filtered_df = filtered_df.dropna()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Generating Document Matrix and Dictionary:

A **document-term matrix** is a mathematical matrix that describes the frequency of terms that occur in a collection of documents. In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms.

A **Dictionary** is collection of all unique tokens present in documents.

In [13]:
def generate_tokens(tweet):
  words=[]
  for word in word_tokenize(tweet):
    # using the if condition because we introduced extra spaces during text cleaning
    if word != '':
       words.append(word)
  return words

#storing the generated tokens in a new column named 'words'
filtered_df['tokens']=filtered_df.rmsContent.apply(generate_tokens)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
filtered_df.tokens

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0          [look, speak, worker, start, ai, model, help, ...
1                              [ask, damn, self, want, read]
2                                      [mute, kindness, ask]
3          [zero, ai, marketing, thing, datum, neural, ma...
4          [end, mix, detail, news, conference, sound, li...
                                 ...                        
1196065                                           [try, wow]
1196066    [thwart, attempt, malicious, javascript, injec...
1196067                                         [vs, riddle]
1196068    [launch, new, ai, system, optimize, dialogue, ...
1196069    [try, talk, new, ai, system, optimize, dialogu...
Name: tokens, Length: 1196070, dtype: object

In [ ]:
# save
filtered_df.to_csv('filtered_verified_tweets.csv', ignore_index=True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# load
filtered_df = pd.read_csv('filtered_verified_tweets.csv')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
def create_dictionary(words):
  return corpora.Dictionary(words)

#passing the dataframe column having tokens as the argument
id2word = create_dictionary(filtered_df.tokens)
print(id2word)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dictionary<29653 unique tokens: ['ai', 'anonymously', 'boss', 'happy', 'help']...>


In [18]:
filtered_df

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,date,username,rawContent,retweetCount,likeCount,lemmaContent,processedContent,rmsContent,tokens
0,1640881210586193921,2023-03-29,cameronwilson,I'm looking to speak to workers who've started...,1,1,look speak worker start chatgpt ai model help ...,i am looking to speak to workers who have star...,look speak worker start ai model help job tell...,"[look, speak, worker, start, ai, model, help, ..."
1,1640880822990557184,2023-03-29,katiewalshstx,"""I asked ChatGPT to-"" keep to your damn self, ...",0,2,ask chatgpt damn self want read,i asked chatgpt to keep to your damn self ...,ask damn self want read,"[ask, damn, self, want, read]"
2,1640880099749937152,2023-03-29,katiewalshstx,"I have to mute ""ChatGPT,"" with kindness, I cou...",3,18,mute chatgpt kindness fuck ask,i have to mute chatgpt with kindness i cou...,mute kindness ask,"[mute, kindness, ask]"
3,1640879874473689089,2023-03-29,martypartymusic,@Blacklist_Ray No there is zero AI in chatGPT ...,0,0,zero ai chatgpt marketing thing datum ...,no there is zero ai in chatgpt its a market...,zero ai marketing thing datum neural...,"[zero, ai, marketing, thing, datum, neural, ma..."
4,1640879019020230658,2023-03-29,WRBolen,How the fuck do you end up mixing in details f...,0,2,fuck end mix detail news conference so...,how the fuck do you end up mixing in details f...,end mix detail news conference sound ...,"[end, mix, detail, news, conference, sound, li..."
...,...,...,...,...,...,...,...,...,...,...
1196065,1598036175285276672,2022-11-30,sethbannon,Trying out ChatGPT and wow https://t.co/TQ0t7B...,15,163,try chatgpt wow,trying out chatgpt and wow,try wow,"[try, wow]"
1196066,1598027429909778432,2022-11-30,minimaxir,ChatGPT thwarts my many attempts at malicious ...,22,186,chatgpt thwart attempt malicious javascript in...,chatgpt thwarts my many attempts at malicious ...,thwart attempt malicious javascript injection,"[thwart, attempt, malicious, javascript, injec..."
1196067,1598024131978547200,2022-11-30,minimaxir,ChatGPT vs. riddles https://t.co/YuPYOD0U7o,2,18,chatgpt vs riddle,chatgpt vs riddles,vs riddle,"[vs, riddle]"
1196068,1598015627540635648,2022-11-30,gdb,"Just launched ChatGPT, our new AI system which...",367,2361,launch chatgpt new ai system optimize dialogue...,just launched chatgpt our new ai system which...,launch new ai system optimize dialogue try,"[launch, new, ai, system, optimize, dialogue, ..."


In [16]:
def create_document_matrix(tokens,id2word):
  corpus = []
  ids = []
  for id, text in tokens.iteritems():
    ids.append(id)
    corpus.append(id2word.doc2bow(text))
  return ids, corpus
  
#passing the dataframe column having tokens and dictionary
ids, corpus=create_document_matrix(filtered_df.tokens,id2word)
#print(tweets_df.tokens[0])
#print(corpus[0])

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
len(corpus)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1196070

### LDA Model

reference:[genism.LDA document](https://radimrehurek.com/gensim/models/ldamodel.html)

In [21]:
lda_model = gensim.models.ldamodel.LdaModel(\
corpus=corpus,\
id2word=id2word,\
num_topics=5,\
random_state=100,
)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


save the model

In [ ]:
%pwd

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'/content/drive/MyDrive/BDT SEM2/5005'

In [28]:
from gensim.test.utils import datapath

# Save model to disk.
model_dir = '/content/drive/MyDrive/BDT SEM2/5005/models/'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
file = datapath(model_dir+"lda_model_verified_filtered_topic5")
lda_model.save(file)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Load the model

In [ ]:
from gensim.test.utils import datapath

file = datapath(root+"lda_model_22Dec")
# Load a potentially pretrained model from disk.
lda_model = gensim.models.ldamodel.LdaModel.load(file)

### PyLDAvis visualization

[How to understand the visualization](https://community.alteryx.com/t5/Data-Science/Getting-to-the-Point-with-Topic-Modeling-Part-3-Interpreting-the/ba-p/614992)

In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.118696  0.140429       1        1  27.363112
0     -0.032067 -0.335878       2        1  23.913818
3     -0.299459 -0.057931       3        1  19.400326
2     -0.154004  0.361884       4        1  15.918007
4      0.366834 -0.108504       5        1  13.404737, topic_info=        Term           Freq          Total Category  logprob  loglift
208   google   87548.000000   87548.000000  Default  30.0000  30.0000
0         ai  279405.000000  279405.000000  Default  29.0000  29.0000
13       ask  101061.000000  101061.000000  Default  28.0000  28.0000
562   openai   76114.000000   76114.000000  Default  27.0000  27.0000
39     write  150903.000000  150903.000000  Default  26.0000  26.0000
..       ...            ...            ...      ...      ...      ...
210     week    7229.909149   29387.199302   Topic5  -5.4459   0.6072
246     good    7899.765974   52802.251797   Topic5  -5.3573   0.1098
808  discuss    5562.362071   14541.442493   Topic5  -5.7081   1.0486
232  teacher    5532.120484   15929.411843   Topic5  -5.7135   0.9520
381    world    5574.693918   34194.635643   Topic5  -5.7059   0.1957

[327 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
147       2  0.927104  actually
147       5  0.072825  actually
0         1  0.617561        ai
0         2  0.000057        ai
0         3  0.095506        ai
...     ...       ...       ...
887       3  0.356504      year
887       4  0.085761      year
887       5  0.037083      year
689       2  0.999884       yes
1653      5  0.999903      york

[519 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3, 5])

In [24]:
# show_topic()

def get_lda_topics(model, num_topics, top_n_words):
  word_dict = {}
  for i in range(num_topics):
    word_dict['Topic #'+ '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn = top_n_words)];
  return pd.DataFrame(word_dict)
  
get_lda_topics(lda_model,5,10)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic #01,Topic #02,Topic #03,Topic #04,Topic #05
0,write,ai,google,model,school
1,ask,tool,ai,user,ban
2,good,like,openai,language,student
3,like,new,microsoft,ai,exam
4,answer,use,search,like,pass
5,think,write,chatbot,datum,amp
6,thing,intelligence,new,use,university
7,know,artificial,bing,month,game
8,question,human,bard,amp,new
9,people,future,company,time,test


### post processing

In [25]:
from operator import itemgetter
from collections import Counter

t=[]

for i in corpus:
    topic_l = lda_model.get_document_topics(i, minimum_probability=None, minimum_phi_value=None, per_word_topics=False)
    t.append(max(topic_l,key=itemgetter(1))[0])

counter = Counter(t)
print(dict(counter))

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{1: 333543, 0: 349898, 4: 129720, 3: 198674, 2: 184235}


In [26]:
len(t)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1196070

In [27]:
filtered_df['topic'] = t
filtered_df

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,date,username,rawContent,retweetCount,likeCount,lemmaContent,processedContent,rmsContent,tokens,topic
0,1640881210586193921,2023-03-29,cameronwilson,I'm looking to speak to workers who've started...,1,1,look speak worker start chatgpt ai model help ...,i am looking to speak to workers who have star...,look speak worker start ai model help job tell...,"[look, speak, worker, start, ai, model, help, ...",1
1,1640880822990557184,2023-03-29,katiewalshstx,"""I asked ChatGPT to-"" keep to your damn self, ...",0,2,ask chatgpt damn self want read,i asked chatgpt to keep to your damn self ...,ask damn self want read,"[ask, damn, self, want, read]",0
2,1640880099749937152,2023-03-29,katiewalshstx,"I have to mute ""ChatGPT,"" with kindness, I cou...",3,18,mute chatgpt kindness fuck ask,i have to mute chatgpt with kindness i cou...,mute kindness ask,"[mute, kindness, ask]",1
3,1640879874473689089,2023-03-29,martypartymusic,@Blacklist_Ray No there is zero AI in chatGPT ...,0,0,zero ai chatgpt marketing thing datum ...,no there is zero ai in chatgpt its a market...,zero ai marketing thing datum neural...,"[zero, ai, marketing, thing, datum, neural, ma...",1
4,1640879019020230658,2023-03-29,WRBolen,How the fuck do you end up mixing in details f...,0,2,fuck end mix detail news conference so...,how the fuck do you end up mixing in details f...,end mix detail news conference sound ...,"[end, mix, detail, news, conference, sound, li...",0
...,...,...,...,...,...,...,...,...,...,...,...
1196065,1598036175285276672,2022-11-30,sethbannon,Trying out ChatGPT and wow https://t.co/TQ0t7B...,15,163,try chatgpt wow,trying out chatgpt and wow,try wow,"[try, wow]",4
1196066,1598027429909778432,2022-11-30,minimaxir,ChatGPT thwarts my many attempts at malicious ...,22,186,chatgpt thwart attempt malicious javascript in...,chatgpt thwarts my many attempts at malicious ...,thwart attempt malicious javascript injection,"[thwart, attempt, malicious, javascript, injec...",0
1196067,1598024131978547200,2022-11-30,minimaxir,ChatGPT vs. riddles https://t.co/YuPYOD0U7o,2,18,chatgpt vs riddle,chatgpt vs riddles,vs riddle,"[vs, riddle]",2
1196068,1598015627540635648,2022-11-30,gdb,"Just launched ChatGPT, our new AI system which...",367,2361,launch chatgpt new ai system optimize dialogue...,just launched chatgpt our new ai system which...,launch new ai system optimize dialogue try,"[launch, new, ai, system, optimize, dialogue, ...",1


In [30]:
tweets_with_topic = filtered_df[['id','date','username','rawContent','topic','retweetCount','likeCount']]
tweets_with_topic

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,date,username,rawContent,topic,retweetCount,likeCount
0,1640881210586193921,2023-03-29,cameronwilson,I'm looking to speak to workers who've started...,1,1,1
1,1640880822990557184,2023-03-29,katiewalshstx,"""I asked ChatGPT to-"" keep to your damn self, ...",0,0,2
2,1640880099749937152,2023-03-29,katiewalshstx,"I have to mute ""ChatGPT,"" with kindness, I cou...",1,3,18
3,1640879874473689089,2023-03-29,martypartymusic,@Blacklist_Ray No there is zero AI in chatGPT ...,1,0,0
4,1640879019020230658,2023-03-29,WRBolen,How the fuck do you end up mixing in details f...,0,0,2
...,...,...,...,...,...,...,...
1196065,1598036175285276672,2022-11-30,sethbannon,Trying out ChatGPT and wow https://t.co/TQ0t7B...,4,15,163
1196066,1598027429909778432,2022-11-30,minimaxir,ChatGPT thwarts my many attempts at malicious ...,0,22,186
1196067,1598024131978547200,2022-11-30,minimaxir,ChatGPT vs. riddles https://t.co/YuPYOD0U7o,2,2,18
1196068,1598015627540635648,2022-11-30,gdb,"Just launched ChatGPT, our new AI system which...",1,367,2361


In [31]:
tweets_with_topic.to_csv('tweets_with_topic.csv')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
topic_df = filtered_df[['date','topic']]
topic_df

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,topic
0,2023-03-29,1
1,2023-03-29,0
2,2023-03-29,1
3,2023-03-29,1
4,2023-03-29,0
...,...,...
1196065,2022-11-30,4
1196066,2022-11-30,0
1196067,2022-11-30,2
1196068,2022-11-30,1


In [45]:
tmp = topic_df.groupby(['date', 'topic']).size().reset_index(name='topic_count')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
tmp

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,topic,topic_count
0,2022-11-30,0,216
1,2022-11-30,1,54
2,2022-11-30,2,36
3,2022-11-30,3,18
4,2022-11-30,4,54
...,...,...,...
675,2023-04-14,0,5
676,2023-04-14,1,2
677,2023-04-14,2,2
678,2023-04-14,3,4


In [47]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         680 non-null    object
 1   topic        680 non-null    int64 
 2   topic_count  680 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 16.1+ KB


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
tmp = tmp.astype({"topic": str,"date":'datetime64[ns]'}, errors='raise') 

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
tmp['topic'] = tmp.topic.astype(str)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
tmp.to_csv('tmp.csv', index=False)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
